## 18.2 Peewee

> _Peewee is a simple and small ORM. It has few (but expressive) concepts, making it easy to learn and intuitive to use._

We'll cover how to:

* Define a Model
* Create a Database
* Store Data
* Update Data
* Remove Data
* Learn About Autocommit
* Retrieve Data

First, let's install Peewee.

In [ ]:
!pip install peewee

### Model Definition

When starting a project with Peewee, it’s typically best to begin with your data model, by defining one or more `Model` classes. This usually goes into the __`models.py`__ file of your application. We'll use SQLite:

In [ ]:
import peewee

db = peewee.SqliteDatabase('peewee.db')  # This ideally belongs in a settings.py file


class BaseModel(peewee.Model):

    class Meta:
        database = db


class User(BaseModel):
    name = peewee.CharField()
    birthday = peewee.DateField(null=True)
    is_relative = peewee.BooleanField(null=True)

We created a `BaseModel` that uses our SQLite database so all models using this inherits this setting.

### Creating the Database

Now that we have our model, let’s connect it to the database. Although it’s not necessary to open the connection explicitly, it is good practice since it will reveal any errors with your database connection immediately, as opposed to some arbitrary time later when the first query is executed. It is also good to close the connection when you are done – for instance, a web app might open a connection when it receives a request, and close the connection when it sends the response.

We’ll begin by creating the tables in the database that will store our data. This code will create the tables with the appropriate columns, indexes, sequences, and foreign key constraints. Your application doesn't need to run this all the time. This is only executed to create the required tables so this should be in a separate script.

In [ ]:
models = [
    User,
]
db.connect()
db.create_tables([User])
db.commit()
db.close()

### Storing data

Let’s begin by populating the database with some people. We will use the __`save()`__ and __`create()`__ methods to add and update people’s records.

You can add a User by calling the save() method on an instance:

In [ ]:
from datetime import date

uncle_bob = User(name='Bob', birthday=date(1960, 1, 15), is_relative=True)
uncle_bob.save()

You can also add a User by calling the create() method, which returns an instance:

In [ ]:
grandma = User.create(name='Grandma', birthday=date(1935, 3, 1), is_relative=True)
herb = User.create(name='Herb', birthday=date(1950, 5, 5), is_relative=False)

### Updating Data

Updating is as simple as updating an instance and the calling __`save()`__ method on it:

In [ ]:
grandma.name = 'Grandma L.'
grandma.save()  # Update grandma's name in the database.

### Removing Data

Removing data is as simple as calling the __`delete_instance()`__ method:

In [ ]:
herb.delete_instance()

### Autocommit

While Peewee initializes databases with __`autocommit=True`__ by default, it has support for transaction handling with `Database.atomic()` or `Database.transaction()` either as a context manager or a decorator. Peewee has no "sessions" like in SQLAlchemy.

#### Context Manager

In [ ]:
db = peewee.SqliteDatabase('peewee.db')

with db.atomic() as txn:
    User.create(name='Charlie')  # This is the outer-most level, so this block corresponds to a transaction.

    with db.atomic() as nested_txn:
        User.create(name='Huey')  # This block corresponds to a savepoint.
        nested_txn.rollback()  # This will roll back the above create() query.

    User.create(name='Mickey')

# When the block ends, the transaction is committed (assuming no error
# occurs). At that point there will be two users, "charlie" and "mickey".

#### Decorator

In [ ]:
@db.atomic()
def create_user(name):
    # This statement will run in a transaction. If the caller is already
    # running in an `atomic` block, then a savepoint will be used instead.
    return User.create(name=name)

create_user('Charlie')

This setting can be turned on and off at runtime anytime you like. If you choose to do so, transactions must be made explicit using `Database.begin()` to begin a transaction, and `Database.commit()` or `Database.rollback()`.

In [ ]:
db.set_autocommit(False)
db.begin()
try:
    herb.delete_instance(recursive=True)
except:
    db.rollback()
    raise
else:
    try:
        db.commit()
    except:
        db.rollback()
        raise
finally:
    db.set_autocommit(True)

Manuall control of every transaction is also available by turning autocommit off when instantiating your database:

In [ ]:
db = peewee.SqliteDatabase('peewee.db', autocommit=False)

db.begin()
User.create(name='Somebody')
db.commit()

### Retrieving Data

The real strength of our database is in how it allows us to retrieve data through queries. Relational databases are excellent for making ad-hoc queries.

#### Getting single records

Let’s retrieve Grandma’s record from the database. To get a single record from the database, use `SelectQuery.get()` or the equivalent shorthand `Model.get()`:

In [ ]:
grandma = User.select().where(User.name=='Grandma L.').get()

# is equivalent to

grandma = User.get(User.name=='Grandma L.')

#### Lists of records

Let’s list all the people in the database:

In [ ]:
for user in User.select():
    print(user.name, user.is_relative)

Let’s list all the people now, youngest to oldest:

In [ ]:
for user in User.select().order_by(User.birthday.desc()):
    print(user.name, user.birthday)

### Summary

Our sample Peewee code may be organized and summed up like this:

In [ ]:
# In your settings file
import peewee

db = peewee.SqliteDatabase('peewee.db')  # This ideally belongs in a settings.py file


# In your models file
class BaseModel(peewee.Model):

    class Meta:
        database = db


class User(BaseModel):
    name = peewee.CharField()
    birthday = peewee.DateField()
    is_relative = peewee.BooleanField()


# In your database creation script
models = [
    User,
]
db.connect()
db.create_tables(models)


# In your controller - creating
from datetime import date

uncle_bob = User(name='Bob', birthday=date(1960, 1, 15), is_relative=True)
uncle_bob.save()

# In your controller - querying
bob = User.get(User.name=='Bob')